In [158]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [159]:
data = pd.read_csv('NED_DATA.csv', skiprows=12)#,dtype=float,skiprows=12)
data.head(8) #showing first 8 values bc that is where the first NaN appears in 'Hubble const.'

,FRN,Exclusion Code,D,G,Galaxy ID,m-M,err,D (Mpc),Method,REFCODE,Unnamed: 10,Unnamed: 11,Hubble const.,Adopted LMC modulus,Date (Yr. - 1980)
0,NaN,1,1,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,SDSS CAND. AGN; MLCS2K2
1,NaN,2,1,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,SDSS CAND. AGN; SALT 2
2,NaN,999999,1,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,"SDSS-II SN 13651, SDSS CAND. AGN; MLCS2K2"
3,NaN,999999,1,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,"SDSS-II SN 13651, SDSS CAND. AGN; SALT 2"
4,NaN,3,2,2MASX J00000138+1530350,38.86,0.46,592.0,FP,2016A&A...596A..14S,NaN,NaN,70.0,NaN,36.0,NaN
5,NaN,4,3,2MASX J00000155-0929403,39.79,0.46,910.0,FP,2016A&A...596A..14S,NaN,NaN,70.0,NaN,36.0,NaN
6,NaN,5,4,UGC 12889,33.65,0.66,53.7,Tully-Fisher,1984A&AS...56..381B,NaN,NaN,103.0,NaN,4.0,Diameter
7,NaN,6,4,UGC 12889,34.16,0.40,68.0,Tully-Fisher,2007A&A...465...71T,NaN,NaN,NaN,NaN,27.0,mean


In [160]:
#Need to fix the errors in column headers

data.columns = data.columns[1:].tolist()+['Notes']
data.head(8)

,Exclusion Code,D,G,Galaxy ID,m-M,err,D (Mpc),Method,REFCODE,Unnamed: 10,Unnamed: 11,Hubble const.,Adopted LMC modulus,Date (Yr. - 1980),Notes
0,NaN,1,1,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,SDSS CAND. AGN; MLCS2K2
1,NaN,2,1,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,SDSS CAND. AGN; SALT 2
2,NaN,999999,1,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,"SDSS-II SN 13651, SDSS CAND. AGN; MLCS2K2"
3,NaN,999999,1,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,"SDSS-II SN 13651, SDSS CAND. AGN; SALT 2"
4,NaN,3,2,2MASX J00000138+1530350,38.86,0.46,592.0,FP,2016A&A...596A..14S,NaN,NaN,70.0,NaN,36.0,NaN
5,NaN,4,3,2MASX J00000155-0929403,39.79,0.46,910.0,FP,2016A&A...596A..14S,NaN,NaN,70.0,NaN,36.0,NaN
6,NaN,5,4,UGC 12889,33.65,0.66,53.7,Tully-Fisher,1984A&AS...56..381B,NaN,NaN,103.0,NaN,4.0,Diameter
7,NaN,6,4,UGC 12889,34.16,0.40,68.0,Tully-Fisher,2007A&A...465...71T,NaN,NaN,NaN,NaN,27.0,mean


In [161]:
data.rename(columns={'Unnamed: 11': 'redshift (z)', 'Unnamed: 10': 'SNID'}, inplace=True)
data.head(8)
#Now we fixed the dataframe

,Exclusion Code,D,G,Galaxy ID,m-M,err,D (Mpc),Method,REFCODE,SNID,redshift (z),Hubble const.,Adopted LMC modulus,Date (Yr. - 1980),Notes
0,NaN,1,1,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,SDSS CAND. AGN; MLCS2K2
1,NaN,2,1,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,SDSS CAND. AGN; SALT 2
2,NaN,999999,1,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,"SDSS-II SN 13651, SDSS CAND. AGN; MLCS2K2"
3,NaN,999999,1,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,2018PASP..130f4002S,SDSS-II SN 13651,0.249917,70.0,NaN,34.0,"SDSS-II SN 13651, SDSS CAND. AGN; SALT 2"
4,NaN,3,2,2MASX J00000138+1530350,38.86,0.46,592.0,FP,2016A&A...596A..14S,NaN,NaN,70.0,NaN,36.0,NaN
5,NaN,4,3,2MASX J00000155-0929403,39.79,0.46,910.0,FP,2016A&A...596A..14S,NaN,NaN,70.0,NaN,36.0,NaN
6,NaN,5,4,UGC 12889,33.65,0.66,53.7,Tully-Fisher,1984A&AS...56..381B,NaN,NaN,103.0,NaN,4.0,Diameter
7,NaN,6,4,UGC 12889,34.16,0.40,68.0,Tully-Fisher,2007A&A...465...71T,NaN,NaN,NaN,NaN,27.0,mean


In [162]:
#I'm first going to drop columns that do not help such as Exclusion code, REFCODE, SNID, Adopted LMC modulus and Notes
'''reason for dropping each
Exclusion code - Tells us if the data is from sources incorporated with NED which is not going to tell us about the measurement
D - We want our decision tree to be based on methods and results not the actual objects
G - Same as D they both use a number to describe the object
REFCODE - Tells what paper the measurement is from
SN_ID - Only relevant for supernova measurements which is not all of our datapoints
Adopted LMC modulus - Similar to SNID only relevant to certain measurements
Notes - Extra infor about measurement not present in most
'''
data = data.drop(['Exclusion Code','D','G','REFCODE','SNID','Adopted LMC modulus','Notes'], axis=1)
#We are looking at whether or not there is a hubble const measurement so we don't want to remove those
data['Hubble const.'] = data['Hubble const.'].fillna(0.0)
data.head(8)

,Galaxy ID,m-M,err,D (Mpc),Method,redshift (z),Hubble const.,Date (Yr. - 1980)
0,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,0.249917,70.0,34.0
1,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,0.249917,70.0,34.0
2,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,0.249917,70.0,34.0
3,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,0.249917,70.0,34.0
4,2MASX J00000138+1530350,38.86,0.46,592.0,FP,NaN,70.0,36.0
5,2MASX J00000155-0929403,39.79,0.46,910.0,FP,NaN,70.0,36.0
6,UGC 12889,33.65,0.66,53.7,Tully-Fisher,NaN,103.0,4.0
7,UGC 12889,34.16,0.40,68.0,Tully-Fisher,NaN,0.0,27.0


In [163]:
data = data.dropna()
hubble = data['Hubble const.']
print(data.shape)
print(hubble.shape)
data.head(8)

(30116, 8)
(30116,)


,Galaxy ID,m-M,err,D (Mpc),Method,redshift (z),Hubble const.,Date (Yr. - 1980)
0,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,0.249917,70.0,34.0
1,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,0.249917,70.0,34.0
2,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,0.249917,70.0,34.0
3,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,0.249917,70.0,34.0
16,SDSS-II SN 14480,41.20,0.33,1300.0,SNIa,0.330796,70.0,34.0
17,SDSS-II SN 14480,41.30,0.32,1370.0,SNIa,0.330796,70.0,34.0
21,SDSS-II SN 14480,41.20,0.33,1300.0,SNIa,0.330796,70.0,34.0
22,SDSS-II SN 14480,41.30,0.32,1370.0,SNIa,0.330796,70.0,34.0


In [164]:
print(hubble.unique())
#there is an odd value coming out here that is 69.7 100. maybe its just printing weirdly
for x in hubble.unique():
    print(x)
#yes just printing weirdly on my screen

[ 70.   74.2   0.   72.   73.8  65.   60.   73.   74.   69.7 100.   71.
  70.8  71.6]
70.0
74.2
0.0
72.0
73.8
65.0
60.0
73.0
74.0
69.7
100.0
71.0
70.8
71.6


In [165]:
#I want to have a column that is either 1 or 0 for 'has hubble measurement or doesnt. So i realize I didn't really need to pull out this 'hubble' variable
hubble2 = np.copy(hubble) #It was modifying the Hubble const column in the df
hubble2[hubble2 != 0.0] = 1.0
#Then stick it back in the df
data['Has Hubble'] = hubble2
data.head(8)

,Galaxy ID,m-M,err,D (Mpc),Method,redshift (z),Hubble const.,Date (Yr. - 1980),Has Hubble
0,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,0.249917,70.0,34.0,1.0
1,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,0.249917,70.0,34.0,1.0
2,SDSS-II SN 13651,41.64,0.17,1700.0,SNIa SDSS,0.249917,70.0,34.0,1.0
3,SDSS-II SN 13651,42.10,0.21,2110.0,SNIa SDSS,0.249917,70.0,34.0,1.0
16,SDSS-II SN 14480,41.20,0.33,1300.0,SNIa,0.330796,70.0,34.0,1.0
17,SDSS-II SN 14480,41.30,0.32,1370.0,SNIa,0.330796,70.0,34.0,1.0
21,SDSS-II SN 14480,41.20,0.33,1300.0,SNIa,0.330796,70.0,34.0,1.0
22,SDSS-II SN 14480,41.30,0.32,1370.0,SNIa,0.330796,70.0,34.0,1.0


This concludes cleaning the data

In [166]:
#Attempt at using ChatGPT to build the tree
#It suggests dropping the columns GalaxyID and Hubble const. both of which I had considered dropping
#Galaxy ID makes sense to drop because it can simply be used as an identifier for the measurement
#Hubble const. makes sense to drop because it is what we are trying to predict
data = data.drop(columns=["Galaxy ID", "Hubble const."])
#It also wants to one hot encode the method column which is fine because there are not a crazy amount of differnt methods
print(data['Method'].unique())
data["Method"] = data["Method"].astype("category").cat.codes
#Lastly split into Target and feature
X = data.drop(columns=["Has Hubble"]).values
y = data["Has Hubble"].values
print(X.shape, y.shape) #just checking and it looks alright


['SNIa SDSS' 'SNIa' 'SNII optical' 'BCG' 'GRB' 'HII LF' 'D-Sigma'
 'Faber-Jackson' 'G Lens']
(30116, 6) (30116,)


In [167]:
#Create a function for gini impurity
def gini(y):
    """Calculate Gini impurity for a list of labels."""
    classes, counts = np.unique(y, return_counts=True)
    prob_sq = (counts / counts.sum()) ** 2
    return 1 - prob_sq.sum()